In [ ]:
%%writefile vetor.cu
 #include <iostream>
 #include <iomanip>
 #include <cstdlib>
 #include <chrono>
 #include <cstdlib>
 #include <algorithm>
 #include <thrust/device_vector.h>
 #include <thrust/host_vector.h>
 #include <thrust/transform.h>
 #include <thrust/reduce.h>
 #include <thrust/functional.h>
 #include <cmath>
 #include <random>

 using namespace std::chrono;

 void reportTime(const char* msg, steady_clock::duration span) {
     auto ms = duration_cast<milliseconds>(span);
     std::cout << msg << " - levou - " <<
      ms.count() << " milisegundos" << std::endl;
 }

 // CRIE UMA FUNCTOR PARA CALCULAR A SQUARE
 struct Square {
    __host__ __device__
    float operator()(float x) {
      return x * x;
    }
 };

 // IMPLEMENTE O CALCULO DA MAGNITUDE COM THRUST
 float magnitude(thrust::device_vector<float>& v_d) {
     float result;

     thrust::device_vector<float> aux(v_d.size());


     // Elevar ao quadrado cada elemento do vetor
     thrust::transform(v_d.begin(), v_d.end(), aux.begin(), Square());


     // Reducao do vetor pela soma
     result = thrust::reduce(aux.begin(), aux.end(), 0.0, thrust::plus<float>());

     // Extrai a raiz quadrada da soma
     result = sqrt(result);

     return result;
 }

 int main(int argc, char** argv) {
     if (argc != 2) {
         std::cerr << argv[0] << ": numero invalido de argumentos\n";
         std::cerr << "uso: " << argv[0] << "  tamanho do vetor\n";
         return 1;
     }
     int n = std::atoi(argv[1]); //numero de elementos
     steady_clock::time_point ts, te;

     // Faça um  vector em thrust
     thrust::device_vector<float> v_d(n);

     // inicilize o  vector
     ts = steady_clock::now();

     //std::generate(v_d.begin(), v_d.end(), std::rand);
     thrust::fill(v_d.begin(), v_d.end(), 1);


     te = steady_clock::now();
     reportTime("Inicializacao", te - ts);

     // Calcule a magnitude do vetor
     ts = steady_clock::now();
     //float len = magnitude(v_d);
     float len = std::sqrt(thrust::transform_reduce( v_d.begin(), v_d.end(), Square(), 0.0, thrust::plus<float>()));

     te = steady_clock::now();
     reportTime("Tempo para calculo", te - ts);


     std::cout << std::fixed << std::setprecision(4);
     std::cout << "Magnitude : " << len << std::endl;
 }

Writing vetor.cu


In [ ]:
!nvcc -arch=sm_70 -std=c++14 vetor.cu -o vetor

In [ ]:
!./vetor 100

Inicializacao - levou - 0 milisegundos
Tempo para calculo - levou - 0 milisegundos
Magnitude : 10.0000
